# CS 1656 – Introduction to Data Science

## Instructor: Alexandros Labrinidis
## Teaching Assistants: Evangelos Karageorgos, Xiaoting Li, Zi Han Ding

### Additional credits: Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham

## Recitation 3: Data Analysis with Pandas
---
So far we have encountered basic data manipulation with pandas Dataframes including row and column selection, boolean indexing, working with missing values, groupby and aggregate functions such as mean(). But there are many other powerful data manipulation and analysis techniques available in pandas. In this recitation, we will learn some more advanced ways for data anylsis in Python using Dataframes. 

Begin by importing pandas package.

In [ ]:
import pandas as pd

Next load the dataset that we will be playing arround with.

In [ ]:
df = pd.read_csv('coffee-chain.csv')
df.head()

Let's get the subset of the dataframe we need.

In [ ]:
df_small = df[['Area Code','Market', 'Market Size', 'Product', 'Product Line', 'Product Type', 'State', 'Type', 'Profit','Total Expenses']].copy()
df_small.head()

## Slicing & Indexing
What we saw above was slicing. Slicing uses the [] operator selects a set of rows and/or columns from a DataFrame.

**Slicing rows**

To slice out a set of rows, you use the following syntax: data[start:stop]. When slicing in pandas the start bound is included in the output. 

In [ ]:
df_small[0:3]

**Slicing vs Copying**

We might have thought that we were creating a fresh copy of df_small when we did slicing. However the statement y = x doesn’t create a copy of our DataFrame. It creates a new variable y that refers to the same object x refers to. This means that there is only one object (the DataFrame), and both x and y refer to it. To create a fresh copy of the DataFrame you can use the syntax y=x.copy(). We will see the effect of slicing but not copying in later steps. 

** Indexing **

We can select specific ranges of our data in both the row and column directions using either label or integer-based indexing.

* loc: indexing via labels or integers or mixed. Referemces rows uning the index.
* iloc: indexing via integers only. References rows using 0-based numbering.

To select a subset of rows AND columns from our DataFrame, we can use the iloc method. For example, 

In [ ]:
df_small.loc[0:3, 'Market': 'Product']

In [ ]:
df_small.iloc[0:4, 1:4]

Notice that indexing in loc is inclusive whereas indexing in iloc is exlusive of the end index

## Statistical Techniques
### Cross-tabulation
Cross tabultaion computes a frequency table of two or more factors. Let's start by making a cross-tab with two variables first.

In [ ]:
df_crosstab = pd.crosstab(df_small["Market"],df_small["Market Size"],margins=False)
df_crosstab

Let'c check the type of the cross-tab

In [ ]:
type(df_crosstab)

Now let's check the value counts of one of our cross-tab's dimensions and see if the totals match?

In [ ]:
pd.value_counts(df_small['Market Size'])

Now let's make a cross-tab with three variables. 

In [ ]:
pd.crosstab(df["Product Type"], [df["Market"],df["Market Size"]],margins=True)

### Binning Data

We can bin our data into categories by specifying bin widths. Let's define equal width bins as shown below. The bins array specifies 4 bins from -800 to -400, -400 to 0, 0 to 400, 400 to 800. We will also specify a group names to assign as labels to each of our bins later. 

In [ ]:
bins = [-800,-400, 0, 400, 800]
group_names = ['Low', 'Okay', 'Good', 'Great']

Now lets bin the data into the categories and add it as a column to the dataframe

In [ ]:
df_small['Categories'] = pd.cut(df_small['Profit'], bins=bins, labels=group_names)
df_small.head(20)

To find out the value counts for each bin of category, we can use value_counts like we did earlier. 

In [ ]:
pd.value_counts(df_small['Categories'])

The result is a Series with 4 values, one for every category. Notice that when printing it, we get both the category name and the value for each category. This is because the series isn't just a list of values. Every value is matched to a category, so its more like an ordered dictionary than a list. The category is the index to the values. If you just selected a column from the original dataframe you would get a Series with the row id as the index, so you would get value 0, value 1 and so on. Using operations like value_counts or group_by, you get series and dataframes that use the groups as the index.

### Quantiles

Pandas allows an easy way of computing percentiles or quartiles. Let's first specify the quantiles we want to calculate,

In [ ]:
quants = [0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0]

To compute the quantiles of Profit and Total Expenses, 

In [ ]:
q = df_small[['Profit','Total Expenses']].quantile(quants)
q

The result here is a dataframe that uses the quantiles as the index of the rows.

### Groupby & Apply

Groupby allows grouping or clustering the dataframe by a particular categorical attribute. 
Apply can be used to apply a function to a group or the entire dataframe. Let's first define the function that we want to apply,

In [ ]:
def get_stats(group):
    print(type(group))
    x= {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean(), 'sum': group.sum()}
    print(x)
    return x

This can be applied to a Dataframe or a grouping of the dataframe as shown below

In [ ]:
df_group = df_small['Profit'].groupby(df_small['Categories']).apply(get_stats)
df_group


The result is a compound Series that has the category as the index, and each value is a dictionary. We can unwind those dictionaries and create a dataframe by using the unstack() function as shown below. 

In [ ]:
df_group.unstack()

### Sorting

Pandas allows nested sorting over mutliple columns of the Dataframe easily as shown below. 

In [ ]:
data_sorted = df_small.sort_values(['Total Expenses', 'Profit'], ascending=False)
data_sorted[['Total Expenses','Profit']].head(20)

## Tasks
For your tasks, use the data file bank-data.csv.

In [ ]:
df = pd.read_csv('bank-data.csv')
df.head()

**Task 1**
Compute the mean income of males versus females. Return a Series of mean income as the values for all sexes.

**Task 2**
Create a cross-tab of save_acct and mortgage. Include the margins.

**Task 3**
Convert the frequencies in task 2's cross-tab to percentages. Include the margins. (Hint: You can use apply)